In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("./../..")

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from effcn.models_mnist import BackboneHinton
from effcn.functions import squash_hinton

import tensorflow as tf

In [5]:
a = torch.rand(3,1,28,28)
a.shape

torch.Size([3, 1, 28, 28])

In [6]:
BH = BackboneHinton()

In [7]:
x = BH(a)
x.shape


#add 1 dimension
#x = torch.unsqueeze(x, dim=1)

x.shape

torch.Size([3, 256, 20, 20])

In [8]:
def capsule_conv(input_tensor,
                      input_dim,
                      output_dim,
                      layer_name,
                      input_atoms=8,
                      output_atoms=8,
                      stride=2,
                      kernel_size=5,
                      padding='same', #"valid" in settings
                      **routing_args):
    
    input_shape = input_tensor.shape

    input_tensor_reshaped = torch.reshape(input_tensor, [
        input_shape[0] * input_dim, input_atoms, input_shape[2], input_shape[3]
    ])

    input_tensor_reshaped.view(-1,input_atoms, input_shape[2], input_shape[3]) 

    return input_shape , input_tensor_reshaped.shape

capsule_conv(input_tensor = x,
                      input_dim =1,
                      output_dim=32,
                      input_atoms=256,
                      output_atoms=8,
                      layer_name='bla'
)

(torch.Size([3, 256, 20, 20]), torch.Size([3, 256, 20, 20]))

In [9]:
# tensor 4d belassen. die reshapes heben sich auf
# input dim ist bei der initalisierung immer 1 daher weg lassen, sonst nur sinnlose reshapen


In [10]:
class PrimaryCapsLayer(nn.Module):
    """
    

    Args:
    c_in: input channels
    c_out: output capsuls
    d_l: dimension of prime caps
    """
    def __init__(self, c_in, c_out, d_l, kernel_size, stride, padding='valid'):
        super(PrimaryCapsLayer, self).__init__()
        self.conv = nn.Conv2d(c_in, c_out * d_l, kernel_size=kernel_size, stride=stride, padding=padding)
        self.c_in = c_in
        self.c_out = c_out
        self.d_l = d_l

    def forward(self, input):
        out = self.conv(input)
        N, C, H, W = out.size()
        out = out.view(N, self.c_out, self.d_l, H, W)

        # will output N x OUT_CAPS x OUT_DIM
        out = out.permute(0, 1, 3, 4, 2)
        out = out.reshape(out.size(0), -1, out.size(4))

        # squash prime caps
        #out = squash_hinton(out)
        return out

In [11]:
PCL = PrimaryCapsLayer(c_in=256, c_out=32, d_l=8, kernel_size=9, stride=2)

p = PCL(x)
p.shape

torch.Size([3, 1152, 8])

In [12]:
class AgreementRouting(nn.Module):
    def __init__(self, n_l, n_h, n_iter):
        super(AgreementRouting, self).__init__()
        self.n_iter = n_iter
        self.b = nn.Parameter(torch.zeros((n_l, n_h)))

    def forward(self, u_predict):
        v, _ = self.forward_debug(u_predict)
        return v

    def forward_debug(self, u_predict):
        batch_size, n_l, n_h, output_dim = u_predict.size()

        c = F.softmax(self.b, dim=-1)
        s = (c.unsqueeze(2) * u_predict).sum(dim=1)
        v = squash_hinton(s)

        if self.n_iter > 0:
            b_batch = self.b.expand((batch_size, n_l, n_h))
            for r in range(self.n_iter):
                v = v.unsqueeze(1)
                b_batch = b_batch + (u_predict * v).sum(-1)

                c = F.softmax(b_batch.view(-1, n_h), dim=-1).view(-1, n_l, n_h, 1)
                s = (c * u_predict).sum(dim=1)
                v = squash_hinton(s)
        return v, c.squeeze(),s

In [13]:
class CapsLayer(nn.Module):
    def __init__(self, n_l, d_l, n_h, d_h, n_iter=3):
        super(CapsLayer, self).__init__()
        self.d_l = d_l
        self.n_l = n_l
        self.d_h = d_h
        self.n_h = n_h
        self.weights = nn.Parameter(torch.Tensor(n_l, d_l, n_h * d_h))
        self.routing_module = AgreementRouting(n_l, n_h, n_iter)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.n_l)
        self.weights.data.uniform_(-stdv, stdv)

    def forward(self, caps_output):
        v, _ = self.forward_debug(caps_output)
        return v

    def forward_debug(self, caps_output):
        caps_output = caps_output.unsqueeze(2)
        u_predict = caps_output.matmul(self.weights)
        u_predict = u_predict.view(u_predict.size(0), self.n_l, self.n_h, self.d_h)
        v, c = self.routing_module.forward_debug(u_predict)
        return v, c

In [14]:
CL =  CapsLayer(1152,8,10,16)

v, c, s = CL.forward_debug(p)

print(v.shape)
print(c.shape)
print(s.shape)

torch.Size([3, 10, 16])
torch.Size([3, 1152, 10])
torch.Size([3, 10, 16])


In [15]:
def squash_hinton(x):
    lengths2 = x.pow(2).sum(dim=2)
    lengths = lengths2.sqrt()
    x = x * (lengths2 / (1 + lengths2) / lengths).view(x.size(0), x.size(1), 1)
    return x


def squash_func(x, eps=10e-21):
    """
        IN:
            x (b, n, d)
        OUT:
            squash(x) (b, n, d)
    """
    x_norm = torch.norm(x, dim=2, keepdim=True)
    return (1 - 1 / (torch.exp(x_norm) + eps)) * (x / (x_norm + eps))

def squash_te(x, eps=10e-21):
    """
        IN:
            x (b, n, d)
        OUT:
            squash(x) (b, n, d)
    """
    x_norm = torch.norm(x, dim=2, keepdim=True)
    norm_squared = x_norm * x_norm
    return (x / x_norm) *(norm_squared / (1 + norm_squared))

In [16]:
print(squash_hinton(p[:1,:5,:7]))


squash_func(p).shape


print(squash_te(p[:1,:5,:7]))

print(squash_hinton(p[:1,:5,:7])-squash_te(p[:1,:5,:7]))

tensor([[[ 0.0102, -0.0061, -0.0143,  0.0119, -0.0226, -0.0167, -0.0088],
         [ 0.0015, -0.0266, -0.0627,  0.0151, -0.0306, -0.0016,  0.0327],
         [ 0.0038, -0.0117, -0.0244,  0.0119, -0.0492, -0.0030, -0.0236],
         [-0.0082, -0.0073, -0.0182,  0.0164, -0.0013, -0.0044,  0.0184],
         [-0.0158,  0.0087,  0.0055,  0.0009, -0.0446, -0.0050, -0.0155]]],
       grad_fn=<MulBackward0>)
tensor([[[ 0.0102, -0.0061, -0.0143,  0.0119, -0.0226, -0.0167, -0.0088],
         [ 0.0015, -0.0266, -0.0627,  0.0151, -0.0306, -0.0016,  0.0327],
         [ 0.0038, -0.0117, -0.0244,  0.0119, -0.0492, -0.0030, -0.0236],
         [-0.0082, -0.0073, -0.0182,  0.0164, -0.0013, -0.0044,  0.0184],
         [-0.0158,  0.0087,  0.0055,  0.0009, -0.0446, -0.0050, -0.0155]]],
       grad_fn=<MulBackward0>)
tensor([[[ 0.0000e+00,  0.0000e+00,  9.3132e-10, -9.3132e-10,  0.0000e+00,
           0.0000e+00,  9.3132e-10],
         [-1.1642e-10,  0.0000e+00,  7.4506e-09,  0.0000e+00,  1.8626e-09,
       

In [17]:
def _update_routing(votes, biases, logit_shape, num_dims, input_dim, output_dim,
                    num_routing, leaky):
  """Sums over scaled votes and applies squash to compute the activations.

  Iteratively updates routing logits (scales) based on the similarity between
  the activation of this layer and the votes of the layer below.

  Args:
    votes: tensor, The transformed outputs of the layer below.
    biases: tensor, Bias variable.
    logit_shape: tensor, shape of the logit to be initialized.
    num_dims: scalar, number of dimmensions in votes. For fully connected
      capsule it is 4, for convolutional 6.
    input_dim: scalar, number of capsules in the input layer.
    output_dim: scalar, number of capsules in the output layer.
    num_routing: scalar, Number of routing iterations.
    leaky: boolean, if set use leaky routing.

  Returns:
    The activation tensor of the output layer after num_routing iterations.
  """
  votes_t_shape = [3, 0, 1, 2]
  for i in range(num_dims - 4):
    votes_t_shape += [i + 4]
  r_t_shape = [1, 2, 3, 0]
  for i in range(num_dims - 4):
    r_t_shape += [i + 4]
  votes_trans = tf.transpose(votes, votes_t_shape)

  def _body(i, logits, activations):
    """Routing while loop."""
    # route: [batch, input_dim, output_dim, ...]
    if leaky:
      route = _leaky_routing(logits, output_dim)
    else:
      route = tf.nn.softmax(logits, dim=2)
    preactivate_unrolled = route * votes_trans
    preact_trans = tf.transpose(preactivate_unrolled, r_t_shape)
    preactivate = tf.reduce_sum(preact_trans, axis=1) + biases
    activation = _squash(preactivate)
    activations = activations.write(i, activation)
    # distances: [batch, input_dim, output_dim]
    act_3d = tf.expand_dims(activation, 1)
    tile_shape = np.ones(num_dims, dtype=np.int32).tolist()
    tile_shape[1] = input_dim
    act_replicated = tf.tile(act_3d, tile_shape)
    distances = tf.reduce_sum(votes * act_replicated, axis=3)
    logits += distances
    return (i + 1, logits, activations)

  activations = tf.TensorArray(
      dtype=tf.float32, size=num_routing, clear_after_read=False)
  logits = tf.fill(logit_shape, 0.0)
  i = tf.constant(0, dtype=tf.int32)
  _, logits, activations = tf.while_loop(
      lambda i, logits, activations: i < num_routing,
      _body,
      loop_vars=[i, logits, activations],
      swap_memory=True)

  return activations.read(num_routing - 1)

In [18]:
x.shape
h = x.unsqueeze(dim=1).unsqueeze(dim=3)
h = h.view(h.shape[0],h.shape[1],-1,8,h.shape[4],h.shape[5])

h.shape

n = h.detach().numpy()
type(n)


f = tf.convert_to_tensor(n, dtype=tf.float32)
f

2022-03-18 14:51:54.771460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-18 14:51:54.771482: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-18 14:51:54.771894: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(3, 1, 32, 8, 20, 20), dtype=float32, numpy=
array([[[[[[2.05562547e-01, 0.00000000e+00, 1.67738348e-01, ...,
            3.15656006e-01, 4.06663865e-02, 2.71764696e-01],
           [0.00000000e+00, 4.17396486e-01, 1.13643438e-01, ...,
            0.00000000e+00, 7.34993368e-02, 1.27972841e-01],
           [1.24635488e-01, 4.12152484e-02, 3.90417222e-03, ...,
            0.00000000e+00, 2.01945320e-01, 1.69305667e-01],
           ...,
           [0.00000000e+00, 1.37741668e-02, 6.21018708e-02, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 1.83801025e-01, 0.00000000e+00, ...,
            5.11386469e-02, 0.00000000e+00, 2.33286068e-01],
           [1.62259594e-01, 2.77619958e-01, 3.24700773e-01, ...,
            9.27700624e-02, 0.00000000e+00, 0.00000000e+00]],

          [[4.96009029e-02, 1.65488347e-01, 0.00000000e+00, ...,
            2.71745268e-02, 3.55308980e-01, 1.96685582e-01],
           [1.70410976e-01, 9.8079159

In [19]:
def weight_variable(shape, stddev=0.1, verbose=False):
  """Creates a CPU variable with normal initialization. Adds summaries.

  Args:
    shape: list, the shape of the variable.
    stddev: scalar, standard deviation for the initilizer.
    verbose: if set add histograms.

  Returns:
    Weight variable tensor of shape=shape.
  """
  with tf.device('/cpu:0'):
    with tf.name_scope('weights'):
      weights = tf.Variable(
          'weights',
          shape,
          #initial_value=tf.initializers.truncated_normal(
          #    stddev=stddev),
          #dtype=tf.float32
          )
  #variable_summaries(weights, verbose)
  return weights


In [20]:
"""
6D Tensor output of a 2D convolution with shape
      `[batch, input_dim, output_dim, output_atoms, out_height, out_width]`
"""

f.shape


TensorShape([3, 1, 32, 8, 20, 20])

In [21]:
weights = weights = tf.Variable(
          'weights',
          [f.shape[1], 256, f.shape[2] * f.shape[3]])

 biases = tf.Variable(
          'biases',
          [f.shape[2],f.shape[3]]],
          initializer=tf.constant_initializer(0.1),
          dtype=tf.float32)

#biases = variables.bias_variable([output_dim, output_atoms])


#input_shape = tf.shape(input_tensor)
#logit_shape = tf.stack([input_shape[0], input_dim, output_dim])
#activations = _update_routing(
#    votes=votes_reshaped,
#    biases=biases,
#    logit_shape=logit_shape,
#    num_dims=4,
#    input_dim=input_dim,
#    output_dim=output_dim,
#    **routing_args)
weights

IndentationError: unexpected indent (3721374513.py, line 5)

In [52]:
torch.manual_seed(0)
b = torch.rand(2,3)
u = torch.rand(1,2,4)
w = torch.rand(2,4,15)

u_predict = u.matmul(w)
u_predict = u_predict.view(u_predict.size(0), 2, 3, 5)
c = F.softmax(b / 1, dim=-1)
s = (c.unsqueeze(2) * u_predict).sum(dim=1)
v = squash_hinton(s)
print(c.unsqueeze(2).shape)
print(u_predict.shape)
print(s.shape)
print(v.shape)

torch.Size([2, 3, 1])
torch.Size([2, 2, 3, 5])
torch.Size([2, 3, 5])
torch.Size([2, 3, 5])


In [55]:
CL = CapsLayer(2,4,3,5)
CL(u)

ValueError: too many values to unpack (expected 2)